Problem Statement.

You are given several logs, where each log contains a unique ID and timestamp. Timestamp is a string that has the following format: Year:Month:Day:Hour:Minute:Second, for example, 2017:01:01:23:59:59. All domains are zero-padded decimal numbers.

Implement the LogSystem class:

    LogSystem() Initializes the LogSystem object.
    void put(int id, string timestamp) Stores the given log (id, timestamp) in your storage system.
    int[] retrieve(string start, string end, string granularity) Returns the IDs of the logs whose timestamps are within the range from start to end inclusive. start and end all have the same format as timestamp, and granularity means how precise the range should be (i.e. to the exact Day, Minute, etc.). For example, start = "2017:01:01:23:59:59", end = "2017:01:02:23:59:59", and granularity = "Day" means that we need to find the logs within the inclusive range from Jan. 1st 2017 to Jan. 2nd 2017, and the Hour, Minute, and Second for each log entry can be ignored.

 

Example 1:

Input
["LogSystem", "put", "put", "put", "retrieve", "retrieve"]
[[], [1, "2017:01:01:23:59:59"], [2, "2017:01:01:22:59:59"], [3, "2016:01:01:00:00:00"], ["2016:01:01:01:01:01", "2017:01:01:23:00:00", "Year"], ["2016:01:01:01:01:01", "2017:01:01:23:00:00", "Hour"]]
Output
[null, null, null, null, [3, 2, 1], [2, 1]]

Explanation
LogSystem logSystem = new LogSystem();
logSystem.put(1, "2017:01:01:23:59:59");
logSystem.put(2, "2017:01:01:22:59:59");
logSystem.put(3, "2016:01:01:00:00:00");

// return [3,2,1], because you need to return all logs between 2016 and 2017.
logSystem.retrieve("2016:01:01:01:01:01", "2017:01:01:23:00:00", "Year");

// return [2,1], because you need to return all logs between Jan. 1, 2016 01:XX:XX and Jan. 1, 2017 23:XX:XX.
// Log 3 is not returned because Jan. 1, 2016 00:00:00 comes before the start of the range.
logSystem.retrieve("2016:01:01:01:01:01", "2017:01:01:23:00:00", "Hour");

 

Constraints:

    1 <= id <= 500
    2000 <= Year <= 2017
    1 <= Month <= 12
    1 <= Day <= 31
    0 <= Hour <= 23
    0 <= Minute, Second <= 59
    granularity is one of the values ["Year", "Month", "Day", "Hour", "Minute", "Second"].
    At most 500 calls will be made to put and retrieve.

# HashTable - O(1)  put, O(N) retrieve runtime, O(N) space

In [2]:
from typing import List

class LogSystem:

    def __init__(self):
        self.logs = {}
        self.g = {'Year': 0, 'Month': 1, 'Day': 2, 'Hour': 3, 'Minute': 4, 'Second': 5}
        
    def getArray(self, tm: str) -> List[int]:
        timestamp = tm.split(':')
        timestamp = [int(t) for t in timestamp]
        return timestamp

    def put(self, id: int, timestamp: str) -> None:
        
        self.logs[id] = self.getArray(timestamp)

    def retrieve(self, start: str, end: str, granularity: str) -> List[int]:
        start = self.getArray(start)
        end = self.getArray(end)
        result = []
        
        for id_, tm in self.logs.items():
            addId, lowerBound, upperBound = True, False, False
            for i in range(self.g[granularity]+1):
                if not lowerBound: 
                    if tm[i] < start[i]:
                        addId = False
                        break
                    if tm[i] > start[i]: lowerBound = True
                if not upperBound:
                    if tm[i] > end[i]:
                        addId = False
                        break
                    if tm[i] < end[i]: upperBound = True
            
            if addId: result.append(id_)
       
        return result

# Binary Search -O(Log N) put, O(max(Log N, result.length)) retrieve runtime, O(N) space

In [3]:
from typing import List
from bisect import insort, bisect_left, bisect_right
class LogSystem:

    def __init__(self):
        self.g = {'Year': 0, 'Month': 1, 'Day': 2, 'Hour': 3, 'Minute': 4, 'Second': 5}
        self.array = []
        self.id = {}

    def put(self, id: int, timestamp: str) -> None:
        t = tuple(timestamp.split(':'))
        self.id[t] = id
        insort(self.array, t)

    def retrieve(self, start: str, end: str, granularity: str) -> List[int]:
        i = self.g[granularity]
        sl = start.split(':')
        el = end.split(':')
        for j in range(i+1,6):
            sl[j] = '00'
            el[j] = '59'
        st, et = tuple(sl), tuple(el)
        left = bisect_left(self.array, st)
        right = bisect_right(self.array, et)
        ans = []
        for i in range(left,right):
            ans += self.id[self.array[i]],
        return ans

In [5]:
instance = LogSystem()
instance.put(1, "2017:01:01:23:59:59")
instance.put(2, "2017:01:01:22:59:59")
instance.put(3, "2016:01:01:00:00:00")
print(instance.retrieve("2016:01:01:01:01:01", "2017:01:01:23:00:00", "Hour"))

[2, 1]
